<a href="https://colab.research.google.com/github/samato88/5-Plus/blob/main/FilterDataFrame_EAST5Plus_Allocate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

More than 5 retentions w/d allocate project

TO DO:
Exclude non participants<br>
Now at allocate step - can finish this off<br>
Make xlsx by lib <br>
For comparison make a count of overlap by matchkey<br>




In [19]:
#@title Memory and GPU info
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')


Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [20]:
#@title import google drive
from google.colab import drive
drive.mount('/content/drive')
#drive.flush_and_unmount()  # Unmount existing mount - trying since not reading new files
#drive.mount('/content/drive', force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
#@title code to skip cells - %%skip

from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    return

In [22]:
#@title imports, set start time
!pip install xlsxwriter
import sys
import os
import re
import time
from datetime import datetime
from zoneinfo import ZoneInfo
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import csv
import numpy as np # Keep this import in the notebook as well
from openpyxl.utils.dataframe import dataframe_to_rows

# Convert the notebook to a .py file and import a module
!jupyter nbconvert --to script /content/drive/MyDrive/Colab\ Notebooks/Balance_North_South.ipynb --output /content/drive/MyDrive/Colab\ Notebooks/Balance_North_South.py


sys.path.append('/content/drive/MyDrive/Colab Notebooks/')
# Import from the converted .py file
from Balance_North_South import balance_north_south

start_time = datetime.now(tz=ZoneInfo('America/Los_Angeles'))
print(f"Starting at: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/Balance_North_South.ipynb to script
[NbConvertApp] Writing 5208 bytes to /content/drive/MyDrive/Colab Notebooks/Balance_North_South.py.txt
Starting at: 2025-08-08 10:02:20


In [23]:
#@title Is this a test run and file locs and global vars (max retain)
testing = False
cloud = True
maxretain = 5

if not cloud:
    path = "/Users/samato/Dropbox/EAST/Gold_Rush/Comparisons/Scripts/"
    inputfiledir = "/Users/samato/Dropbox/EAST/Comparisons/MoreThanFiveEast-2023/" ;
    libmetadata = "/Users/samato/Dropbox/EAST/Gold_Rush/Comparisons/Scripts/Member_Metadata.xlsx";  #

    if testing:
        dir = "/Users/samato/Dropbox/EAST/Gold_Rush/Comparisons/Scripts/Test/"
        outputdir = dir + "Reports/"
        infile = dir + "ForAllFile_test2.csv"
        infilenoocn = dir + "ForALLNoOCNFile-test.csv"
        feedbackcounter = 5
        fields = "/Users/samato/Dropbox/EAST/Gold_Rush/Post-2010-Group-Collection-Analysis-2023/GR_Reports/Fields.xlsx"
        libmetadata = "/Users/samato/Dropbox/EAST/Gold_Rush/Comparisons/Scripts/Member_Metadata.xlsx";  #0:instID, 1:lib, 2:GRLibName, 3:State, 4:Sym, 5:CollSize, 6:Retentions, 7:%, 8:Type, 9:Zone, 10:Region

    else:
        dir = "/Users/samato/Dropbox/EAST/Gold_Rush/Post-2010-Group-Collection-Analysis-2023/GR_Reports/AllFiles/" ;
        outputdir = dir + "Reports/"
        infile = dir + "ALL.csv"
        infilenoocn = dir + "ALL.NoOCN.csv"
        feedbackcounter = 50000
        fields = "/Users/samato/Dropbox/EAST/Gold_Rush/Post-2010-Group-Collection-Analysis-2023/GR_Reports/Fields.xlsx"
        libmetadata = "/Users/samato/Dropbox/EAST/Gold_Rush/Comparisons/Scripts/Member_Metadata.xlsx";  #0:instID, 1:lib, 2:GRLibName, 3:State, 4:Sym, 5:CollSize, 6:Retentions, 7:%, 8:Type, 9:Zone, 10:Region

if cloud:
    path = "/content/drive/MyDrive/Colab Notebooks/5+/"
    libmetadata = "/content/drive/MyDrive/Colab Notebooks/Includes/Member_Metadata.xlsx"; # state, % retained
    fields = "/content/drive/MyDrive/Colab Notebooks/Includes/Fields.xlsx"; # state, % retained

    if testing:
        #filestub = 'GR18Plus'
        filestub = 'EAST5+.test'
        filename = 'ForALLFile_' + filestub + '.csv'
        feedbackcounter = 2
        infile = path + 'Test-InputFiles/' + filename
        outputdir = path + 'Test-Reports/'
        outputallfile = outputdir + 'ALL.csv'
        infilenoocn = path + 'Test/' + 'ForALLNoOCNFile-' + filestub + '.csv'
    else:
        filestub = 'EAST5+'
        filename = 'ForALLFile_' + filestub + '.csv'
        feedbackcounter = 50000
        infile = path + 'InputFiles/' + filename
        outputdir = path + 'Reports/'
        outputallfile = outputdir + 'ALL.csv'
        infilenoocn = path + 'ForALLNoOCNFile-' + filestub + '.csv' # in this case this is the same as the _modified.txt file

print("infile:       ", infile)
print("outputdir:    ", outputdir)
print("outputallfile:", outputallfile)


infile:        /content/drive/MyDrive/Colab Notebooks/5+/InputFiles/ForALLFile_EAST5+.csv
outputdir:     /content/drive/MyDrive/Colab Notebooks/5+/Reports/
outputallfile: /content/drive/MyDrive/Colab Notebooks/5+/Reports/ALL.csv



| LibMetadata                      | Index | Input File PALNI       | Index |
|----------------------------------|-------|--------------------------|--------|
| Inst_ID                          | 0     | A Title                  | 0      |
| Name in Gold Rush Reports EAST  | 1     | B Author                 | 1      |
| State                            | 2     | C Library                | 2      |
| Symbol                           | 3     | D Control Number(001)    | 3      |
| Est Collection Size              | 4     | E ISBN                   | 4      |
| # in Gold Rush (2022)           | 5     | F Publisher Name         | 5      |
| Percent Collection               | 6     | G LC                     | 6      |
| Retention Partners               | 7     | H LC Class               | 7      |
| Census Zone                      | 8     | I Edition                | 8      |
| Region                           | 9     | J Publication Date       | 9      |
| Allocation File Name             | 10    | K Language               | 10     |
| URL                              | 11    | L MatchKey               | 11     |
| FEMA Zone                        | 12    | M 035 field              | 12     |
| Name in Members xlsx             | 13    | N OCN                    | 13     |
| Cohort                           | 14    | O EAST Counts            | 14     |
| OCLC Number Location             | 15    | P PALNI Counts           | 15     |
|                                  |       | Q PALNI Holders          | 16     |
|                                  |       | R Indiana Counts         | 17     |
|                                  |       | S WorldCat Holdings      | 18     |
|                                  |       | T From Sheet             | 19     |
|                                  |       | U Juvenile Lit.          | 20     |
|                                  |       | V HathiTrust             | 21     |
|                                  |       | W Doc: 086               | 22     |
|                                  |       | X Suspect                | 23     |
|                                  |       | Y States Retained In     | 24     |
|                                  |       | Z Held In States         | 25     |
|                                  |       | AA Zone                  | 26     |
|                                  |       | AB Retained By Symbol    | 27     |
|                                  |       | AC Held By Symbols       | 28     |
|                                  |       | AD Region                | 29     |


In [24]:
#@title def updatePercent
def updatePercent(libmetadata_dict, librarytoretain):
    # libname: 0:state,1:symbols,2:collection size 3:retentions,4:%,5:census zone 6:allocated,7:unique,8:topping,9:suspect,10:noOCN
    retentions = libmetadata_dict[librarytoretain][3]
    collectionsize = libmetadata_dict[librarytoretain][2]
    newpercent = int(retentions) / int(collectionsize)
    newpercent = retentions / collectionsize     # calculate new percent
    #print(librarytoretain, "OLD PERCENT:", libmetadata_dict[librarytoretain][4])
    libmetadata_dict[librarytoretain][4] = str(newpercent)  # update percent
    #print("row[2], NEW PERCENT:", libmetadata_dict[librarytoretain][4])

In [25]:
#@title def deassign
def deassign(reduceby, OCN, groupdf):

  # pick which libs get added to daccessdf, allocate N/S has already happened by here, set only contains just north or just south - now looking at tiers

  # row: Pandas(Title='Test Title', Author='Zaehner, R. C.', Library='Bard College', _3='703157', ISBN=394718933.0, _5='Vintage Books', LC='BL65', _7='BL', Edition=nan, _9=1974, Language='English', MatchKey='zendrugsandmysticism_______________________________________________________1974____1__vintaa________________________________________zaehn_______________p', _12=nan, OCN=12345, _14=12345, _15=np.int64(7), _16=3, _17=nan, _18='N', HathiTrust=nan, _20=nan, Suspect=nan, _22=nan, _23=nan, Zone='North', _25='RIU,NKF,RRR,VXW,AUM,AMH,WLU ', _26=nan, Region='Mid Atlantic')
  assert len(groupdf) >= reduceby, "Trying to remove more than we have in Assign function" # something really wrong if this fails - trying to reduce by more than have
  global deaccessdf
  global tier3total
  global totalreduceby # sanity check - did we get all of the reduce by into into this subroutine?

  totalreduceby += reduceby

  ############# this is an alternative to 'merge' below - faster with small datasets like this, slower with large ones
  # choosefrom_df = pd.DataFrame(columns=['Library', 'Percent Category', 'Cohort', 'FEMA Zone'])
  # for row in groupdf.itertuples(index=False): # iterate each library that has this ocn
  #   choosefrom_df['Library'] = row.Library
  #   choosefrom_df['Percent Category'] = libmetadata_dict[row.Library]['Percent Category']
  #   choosefrom_df['Cohort'] = libmetadata_dict[row.Library]['Cohort']
  #   choosefrom_df['FEMA Zone'] = libmetadata_dict[row.Library]['FEMA Zone']
  #   choosefrom_df['Participating'] = libmetadata_dict[row.Library]['Participating']
  ##########

  # merge groupdf with libmetameta_df - here is the dataset we'll use to pick the deaccession winers
  choosefrom_df = groupdf.merge(libmetadata_df, on='Library', how='left')[['Library','Percent Category', 'Cohort', 'FEMA Zone', 'FEMA Random Weight', 'Participating']]
  if testing:
    print("OCN count to start (just N or S here not total OCN count): ", len(groupdf))
    print("CHOOSE FROM:\n" + choosefrom_df['Library'].to_string(index=False, header=False))

# Create Tier1 and check if participating
  tier1_df = choosefrom_df[
      (choosefrom_df['Percent Category'].isin(['High', 'Very High'])) &
      (choosefrom_df['Cohort'].isin(['C1', 'C2', 'M-Florida-2019', 'M-2020', 'M-USMAI-2020'])) &
      (choosefrom_df['Participating'] != 'No')
  ]

  if testing:
   print("Tier 1: ", len(tier1_df), "Reduceby:", reduceby, "\n", tier1_df['Library'].to_string(index=False, header=False))

  # Real selection work starts here:

  if len(tier1_df) == reduceby: # wow, exactly right number of surplus copies
    assign_df = tier1_df
    if testing:
      print("Tier1df => Assigned\n", assign_df['Library'].to_string(index=False, header=False))
    reduceby = reduceby - len(tier1_df) # shouldn't you just say zero?

  elif len(tier1_df) > reduceby: # in tier1 there are plenty to chose from for desassign, pick them
    #assign = tier1_df.sample(n=reduceby)# randomly pick reduceby number of libraries. .sample gets unique picks, but can't be weighted.
    weights = tier1_df['FEMA Random Weight'].values
    probabilities = weights / weights.sum() # Normalize weights - np.random.choice needs values that equal 1
    assign_df = tier1_df.loc[np.random.choice(tier1_df.index, size=reduceby, replace=False, p=probabilities)]
    if testing:
      print("Tier1df => Assigned\n", assign_df['Library'].to_string(index=False, header=False))
    reduceby = reduceby - len(assign_df) # should be zero?

  else: # need more than tier1 - assign all tier one and make tier 2
    assign_df = tier1_df # assign all tier one
    reduceby = reduceby - len(tier1_df) # update reduceby

    choosefrom_df = choosefrom_df[~choosefrom_df['Library'].isin(tier1_df['Library'])]# remove tier one from choosefrom
    if testing:
      print("...At tier 2") # make tier 2
    tier2_df = choosefrom_df[
      (choosefrom_df['Percent Category'].isin(['Medium' or 'Low'])) &
      (choosefrom_df['Cohort'].isin(['C1', 'C2', 'M-Florida-2019', 'M-2020', 'M-USMAI-2020'])) &
      (choosefrom_df['Participating'] != 'No')
    ]
    if testing:
      print("Tier 2: ", len(tier2_df), "Reduceby:", reduceby, "\n", tier2_df['Library'].to_string(index=False, header=False))

    if len(tier2_df) == reduceby: # Tier2 has just exactly what we need
      assign_df = pd.concat([assign_df, tier2_df])
      reduceby = reduceby - len(tier2_df) # should be zero
    elif len(tier2_df) > reduceby: # in tier2 there are plenty to chose from for desassign, pick them
      weights = tier2_df['FEMA Random Weight'].values
      probabilities = weights / weights.sum() # Normalize weights - np.random.choice needs values that equal 1
      assign_df = pd.concat([assign_df, tier2_df.loc[np.random.choice(tier2_df.index, size=reduceby, replace=False, p=probabilities)]])
      reduceby = reduceby - reduceby # why not just say equal 0 ...
    else:
      # assign all tier 2, make a tier 3, make sure all reduceby allocated, and do you need to reset 'assign' each time?
      assign_df = pd.concat([assign_df, tier2_df]) # assign all tier two
      reduceby = reduceby - len(tier2_df) # update reduceby

      choosefrom_df = choosefrom_df[~choosefrom_df['Library'].isin(tier2_df['Library'])]# remove tier two from choosefrom
      if testing:
        print("BUT WAIT, there's more for Tier 3: ", reduceby, "Len of choosefrom:", len(choosefrom_df))
      tier3total = tier3total + reduceby # update tier 3 total to see how many copies fall in this bucket

      #SEA CHECK THIS/UDATE to right formula as you add more tiers, this basically makes it always work for assert - geesh.
      reduceby = reduceby - reduceby

 # foreach assigned library, print the groupdf['Library'] row to a df
  for row in assign_df.itertuples(index=False): # iterate each library assigned and add to deaccess df
    subset = groupdf[groupdf["Library"] == row.Library]
    deaccess_list.append(subset)  # append to the list instead of growing a df
    #deaccessdf = pd.concat([deaccessdf, groupdf[groupdf["Library"] == row.Library]], ignore_index=True) # ineffiecient over time

  assert reduceby == 0, f"REDUCEBY: {reduceby} . Should have been 0" # definitely something wrong if this isn't zero by here - and you'll end up here until you do the "but wait" section above
  if testing:
    print("___")

  return


In [26]:
#@title def Allocate
def allocate():
    count = 0
    lnow = datetime.now(tz=ZoneInfo('America/Los_Angeles')) # initialize

    global totalsurplusaccordingtoalloc

    # Precompute Zone filters once - suggested by ChatGPT for speed
    north_mask = df['Zone'] == 'North'
    south_mask = df['Zone'] == 'South'

    for ocn, group in df.groupby('Current OCN'): # foreach distinct OCN = could alternatively use matchkey here
      count += 1
      if (count) % feedbackcounter == 0: # start timer
        now =  datetime.now(tz=ZoneInfo('America/Los_Angeles'))
        elapsed_time = now - start_time
        interval_time = now - lnow
        #total_minutes = td.total_seconds() // 60  # full minutes including days ; .total_seconds is total, .seconds is in the last day

        print(f"Processed {count} rows, Elapsed seconds: {elapsed_time.total_seconds() /60:.2f}, Interval: {interval_time.total_seconds() /60:.2f}")
        lnow = now

      if testing:
        print(f"\nGroup: {int(ocn)}") # currently working on this OCN
      # figure out how many spare copies there are
      numberofcopies = len(group)

      # SEA HERE
      # this is kinda problematic because this number is based on OCN overlap in GR file, not API overlap.
      #numberSurplus = numberofcopies - maxretain
      # what would happen if we used an EAST Counts from API number here?
      numberSurplus = group['EAST Counts'].iloc[0] - maxretain
      # check if numberSurplus greater than numberofcopies - that would be a problem!
      # opting to use API when that surplus GT group size, and group size otherwise
      if numberSurplus > numberofcopies:
          numberSurplus = numberofcopies - maxretain
      if testing:
          print("number of GR copies:" , numberofcopies, ", API surplus:", numberSurplus)
          #number of copies: 6 , surplus: 1
      assert numberofcopies >= numberSurplus, f"numberSurplus ( usually but not always via API) {numberSurplus} results in a number greater than the GR available group: {numberofcopies} for {ocn}\n {group} "



      totalsurplusaccordingtoalloc += numberSurplus
      if testing:
        print("totalsurplusaccordingtoalloc:", totalsurplusaccordingtoalloc)

      #numberNorth = (group['Zone'] == 'North').sum()
      #numberSouth = (group['Zone'] == 'South').sum()

      # Use masks directly for speed - suggested by ChatGPT as replacement for above
      numberNorth = north_mask[group.index].sum()
      numberSouth = south_mask[group.index].sum()

      if testing:
        print("number of copies:" , numberofcopies, ", surplus:", numberSurplus)
        print(numberNorth, "Held in the North")
        print(numberSouth, "Held in the South")

      #Given 3 variable numberNorth, numberSouth and numberSurplus, determine how many in numberSurplus should be removed from numberNorth and/or numberSouth to make numberNorth and numberSouth be as equal as possible
      reduceNorthby, reduceSouthby, surplus_remaining, numberSouth, numberNorth = balance_north_south(numberNorth, numberSouth, numberSurplus, testing)
      if testing:
        print("Remove North: ", reduceNorthby)
        print("Remove South: ", reduceSouthby)

      # de-allocate - pass in the subgroups
      if reduceNorthby > 0:
        subgroup = group[group['Zone']=='North'] # make a df subgroup of just North
        deassign(reduceNorthby, ocn, subgroup)

      if reduceSouthby > 0:
        subgroup = group[group['Zone']=='South'] # make a df subgroup of just South
        deassign(reduceSouthby, ocn, subgroup)

    return

In [27]:
#@title SKIPPING define dictionaries for dataframes - might not be using libmetadatadict - this might be unncecessary
%%skip
libmetadata_dict = {} # map name to metadata



In [28]:
#@title Read in intial data, drop header rows, create empty deaccess_list, SET GLOBALS
print(infile)
# Define the column types in a dictionary
column_types = {
    'Control Number(001)': 'string',  # For string data
    #'EAST Counts': 'Int64'  # fails if it hits a header row, deleting those below
    'EAST Counts': 'string'
}
df = pd.read_csv(infile, dtype=column_types)
#print(df.columns)

print("df w/ header rows: ", len(df))
Headers_to_drop = df[df['Library'].str.startswith('Library')].index
df.drop(index=Headers_to_drop, inplace=True)
print("df w/o header rows:", len(df))

# EAST counts is #.# or ?? and needs to be int
df['EAST Counts'] = pd.to_numeric(df['EAST Counts'], errors='coerce')
df['EAST Counts'] = df['EAST Counts'].round().astype('Int64')

df['Control Number(001)'] = df['Control Number(001)'].astype('string')

'''
is_duplicate = df.duplicated(subset=["Library", "OCN"])
removed_rows = df[is_duplicate]
print("removed rows:", len(removed_rows), removed_rows[["Library", "OCN"]])
'''
df = df[~df.duplicated(subset=["Library", "OCN"])]

print("df deduped len:", len(df))

deaccess_list = []  # global deaccessdf with this list
tier3total = 0 # global tier 3 count - this eventually will not be necessary
totalreduceby = 0 # just another counter for sanity
totalsurplusaccordingtoalloc = 0


/content/drive/MyDrive/Colab Notebooks/5+/InputFiles/ForALLFile_EAST5+.csv


/tmp/ipython-input-194557436.py:9: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(infile, dtype=column_types)


df w/ header rows:  3017291
df w/o header rows: 3017291
df deduped len: 2998251


In [29]:
#@title SKIPPING Read in field definitions, libmetadata, make states_dict - DON"T THINK I NEED THIS SECTION, except maybe fields
%%skip
# 0:instID, 1:lib, 2:GRLibName, 3:State, 4:Sym, 5:CollSize, 6:Retentions, 7:%, 8:Type, 9:Zone, 10:Region
fields_df = pd.read_excel(fields, sheet_name='Field Definitions 5+', dtype=str) ## Read in field definitions
# states_df = pd.read_excel(libmetadata, sheet_name='Zones', dtype=str) ## Read in state to zone mappings
# states_dict = states_df.set_index('State')['Zone'].to_dict() # Make states to zone dict
# risk_df = pd.read_excel(libmetadata, sheet_name='Risk', dtype=str) ## Read in risk definitions
# risk_dict = risk_df.set_index('Library')['FEMA ZONE'].to_dict() # Make id to risk

# if testing:
#   print(risk_dict)
#   print(states_dict)

In [30]:
#@title Read the EAST Libraries Metadata file, create Low/Medium/High/Very High column
#  I think I have since added a column with low/med etc. in data file, but this does the same...

libmetadata_df = pd.read_excel(libmetadata, sheet_name='Plus5', dtype=str)
libmetadata_df['# in Gold Rush (2022)'] = pd.to_numeric(libmetadata_df['# in Gold Rush (2022)'], errors='coerce').astype(pd.Int64Dtype()) # make these ints
libmetadata_df['Est Collection Size'] = pd.to_numeric(libmetadata_df['Est Collection Size'], errors='coerce').astype(pd.Int64Dtype())
libmetadata_df['FEMA Random Weight'] =  pd.to_numeric(libmetadata_df['FEMA Random Weight'],  errors='coerce').astype(pd.Int64Dtype())



libmetadata_df["Percent Collection"] = pd.to_numeric(libmetadata_df["Percent Collection"], errors="coerce") * 100

conditions = [
    libmetadata_df["Percent Collection"] < 10,
    libmetadata_df["Percent Collection"].between(10, 20, inclusive="both"), # ends at 20
    libmetadata_df["Percent Collection"].between(20, 25, inclusive="both"), # starts at 20.00001 sort of
    libmetadata_df["Percent Collection"] > 25
]

# Define the corresponding values
choices = ["Low", "Medium", "High", "Very High"]

# Apply conditions
libmetadata_df["Percent Category"] = np.select(conditions, choices, default="Unknown")

#print(libmetadata_df[['Name in Gold Rush Reports EAST', 'Percent Collection', 'Percent Category']].to_string(index=False))
#print(libmetadata_df[['Percent Category']].to_string(index=False))

libmetadata_dict = libmetadata_df.set_index('Name in Gold Rush Reports EAST').to_dict(orient='index') # not sure I'm using this anymore?

libmetadata_df = libmetadata_df.rename(columns={"Name in Gold Rush Reports EAST": "Library"})

#print(libmetadata_dict)

In [31]:
#@title SKIP Read in and make df for noocnworksheet
%%skip
column_types = {
    'Control Number(001)': 'string',  # For string data
}
noOCN_df = pd.read_csv(infilenoocn, dtype=column_types) # read the no OCLC numbers infile and make a pandas dataframe - noOCN_df

total_noOCN = len(noOCN_df)

noOCN_df.rename(columns={'OCN': 'OCLC Number', 'States Retained In': 'Retained In State'}, inplace=True)

# another approach would be -> for lib in noOCN_df['Library'].unique():

groups = noOCN_df.groupby('Library')
for lib, row in groups:
  if lib.startswith('SC') or lib.startswith('Library'):
    continue

  #print("Name:", lib, "Row:", row)
  #print(type(row))
  #(row['Title'])
  #print(row['Library'])

  #print(row.iloc[:, [2,3,13,0,1,4,5,9,8,6,7,10,12]])
  rows_to_append = row.iloc[:, [2,3,13,0,1,4,5,9,8,6,7,10,12]]  #
  #print(type(rows_to_append)) #df
  #print( type( dataframes_NoOCN[lib])) #df

  #existing_df = pd.concat([existing_df, rows_to_append], ignore_index=True)
  #dataframes_NoOCN['Anderson University']
  dataframes_NoOCN[lib] = pd.concat([dataframes_NoOCN[lib], rows_to_append], ignore_index=True)


  #dataframes_NoOCN[row['Library']].loc[len(dataframes_NoOCN[row['Library']].index)] = rows_to_append.tolist()
  libmetadata_dict[lib][10] += len(rows_to_append)
  #libmetadata_dict[lib][10] += 1 # increment noOCN


print("items in noocn csv:", len(noOCN_df))
print("libraries in dataframes_NoOCN:", len(dataframes_NoOCN))

#print(len(dataframes_NoOCN['Bard College']))
#print((dataframes_NoOCN['Bard College']))

In [32]:
#@title reduce df to only EAST > 5 (drop less than 6)


print("length of DF before drop EAST < 6:", len(df))
to_drop = df[ (df['EAST Counts'] < 6)  | (df['EAST Counts'].isna()) ].index  # drop if EAST < 6 or is NaNprint(len(to_drop))
df.drop(index=to_drop, inplace=True)
print("length of DF after drop  EAST < 6:", len(df))



length of DF before drop EAST < 6: 2998251
length of DF after drop  EAST < 6: 1515470


In [33]:
#@title Call allocate()
#------------------------------------------------------------------------------------------------------------------------------
print("Starting to allocate multiple copies:", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')) # Convert to a datetime object
print("Number of OCNS needing allocation:", df['Current OCN'].nunique())
if testing:
    print(df['Current OCN'].dropna().unique())
#allocate(df, libmetadata_dict)
allocate()
#------------------------------------------------------------------------------------------------------------------------------

Starting to allocate multiple copies: 2025-08-08 17:03:14
Number of OCNS needing allocation: 236101
Processed 50000 rows, Elapsed seconds: 6.61, Interval: 5.71
Processed 100000 rows, Elapsed seconds: 12.11, Interval: 5.50
Processed 150000 rows, Elapsed seconds: 17.69, Interval: 5.58
Processed 200000 rows, Elapsed seconds: 23.13, Interval: 5.44


In [34]:
#@title Make spreadsheets per lib -
#print(deaccess_list[0]) # deaccess list is a list of dataframes - might want to rethink that...
#print(f"Type of first element: {type(deaccess_list[0])}")

deaccess_df = pd.concat(deaccess_list, ignore_index=True) # concat all the dfs into one

print_columns = ['Library', 'Control Number(001)', 'Title', 'Author', 'ISBN', 'Publisher Name', 'LC', 'LC Class', 'Edition', 'Publication Date', 'Language', '035 field', 'OCN', 'Current OCN']

deaccess_df.to_csv(outputallfile, index=False)

for library, group_df in deaccess_df.groupby('Library'):
    #print(f"Library: {library}")
    outputfile = outputdir + library + ".xlsx"

    with pd.ExcelWriter(outputfile, engine='xlsxwriter') as writer: # create a excel writer object
        workbook  = writer.book
        comma_format = workbook.add_format({'num_format': '[=0]0;[>0]#,###'}) # thanks claude
        text_format = workbook.add_format({'num_format': '@'})

        group_df[print_columns].to_excel(writer, sheet_name="Deassigned", index=False, header=True, freeze_panes=(1,0)) #

        writer.sheets['Deassigned'].set_column('A:B', 20, text_format)
        writer.sheets['Deassigned'].set_column('C:C', 40, text_format)
        writer.sheets['Deassigned'].set_column('D:D', 30, text_format)
        writer.sheets['Deassigned'].set_column('E:E', 15, text_format)
        writer.sheets['Deassigned'].set_column('F:F', 30, text_format)
        writer.sheets['Deassigned'].set_column('G:G', 10, text_format)
        writer.sheets['Deassigned'].set_column('H:H', 6, text_format)
        writer.sheets['Deassigned'].set_column('I:I', 12, text_format)
        writer.sheets['Deassigned'].set_column('J:J', 12, text_format)
        writer.sheets['Deassigned'].set_column('K:K', 12, text_format)
        writer.sheets['Deassigned'].set_column('L:L', 20, text_format)
        writer.sheets['Deassigned'].set_column('M:M', 12, text_format)
        writer.sheets['Deassigned'].set_column('N:N', 12, text_format)

        # Define a format for wrapped text
        header_format = workbook.add_format({'text_wrap': True, 'bold': True, 'bg_color': 'silver'}) # silver is #C0C0C0
        for col_num, value in enumerate(group_df[print_columns].columns.values):         # Apply the format to the header row
            writer.sheets['Deassigned'].write(0, col_num, value, header_format)


In [35]:
#@title Make summary counts sheet

summary_file = outputdir + "Summary.xlsx"

deaccessdf = pd.concat(deaccess_list, ignore_index=True)

summary_counts_df = deaccessdf['Library'].value_counts().reset_index()
summary_counts_df.columns = ['Library', 'Counts']
summary_counts_df = summary_counts_df.sort_values(by="Library").reset_index(drop=True)
print(summary_counts_df)

deaccessdf.to_csv(outputallfile, index=False) # write out fill results file

with pd.ExcelWriter(summary_file, engine='xlsxwriter') as writer2:
    summary_counts_df.to_excel(writer2, sheet_name="Summary Counts", index=False, header=True, freeze_panes=(1,0))
    workbook2  = writer2.book

    countsworksheet2 = writer2.sheets['Summary Counts']
    countsworksheet2.set_column('A:A', 30)  # Column A:A width
    countsworksheet2.set_column('B:F', 10)  # Column B:F width


                             Library  Counts
0                    Adelphi Swirbul   23607
1                    Amherst College   11673
2                       Bard College     763
3                     Boston College   16739
4                  Boston University    2997
5                           Brandeis   22812
6            Bridgewater State Univ.   10847
7                  Bryn Mawr College    7961
8                Bucknell University    1209
9                      Colby College     120
10               Connecticut College    4875
11                  Davidson College     489
12                DeSales University     304
13              Fairfield University   10726
14       Florida Atlantic University     533
15     Florida Gulf Coast University     271
16  Florida International University     592
17          Florida State University     490
18             Frostburg State Univ.    1021
19                Gettysburg College     851
20          Hamilton College Library   29666
21        

In [36]:
#@title summary to screen
print("----Summary---")

# quick summary of how many surplus copies
# (Overlap Level MINUS required number to keep) * Number of MatchKeys or OCNS at that level
# need for calculations:Overlap Level,	Unique Matchkeys or OCNS at that level =>	Number of Copies

counts_df = df['EAST Counts'].value_counts().reset_index()
counts_df.columns = ['EAST', 'Copies']
counts_df['EAST Counts Minus 5'] = counts_df['EAST'] - 5
counts_df['Surplus'] = counts_df['EAST Counts Minus 5'] * (counts_df['Copies']/counts_df['EAST'])
counts_df['Surplus'] = counts_df['Surplus'].astype(int)

#print(counts_df.sort_values(by='EAST'))
print(counts_df.sort_values(by='EAST').to_string(index=False)) # don't include index

# could work for matchkey count too
print('Overall,', df['Current OCN'].nunique(), "titles representing", len(df), "copies had more than 5 copies")
print("\nTotal number of surplus copies according to countsdf  :", counts_df['Surplus'].sum(), "<= will be wrong if some OCN/Matchkey count mismatches")
print("Total number of surplus copies according to allocation:", totalsurplusaccordingtoalloc)
print("Total number that went to deaccession - should be same as surplus:", totalreduceby)
print("\nLen deaccessdf:", len(deaccessdf), "should be the same as surplus copies plus tier 3:", tier3total)

print("TIER 3 total - not assigning", tier3total)


# End the timer and report how long it took
end_time = datetime.now(tz=ZoneInfo('America/Los_Angeles'))
print(f"\nStarted at: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Ending at:  {end_time.strftime('%Y-%m-%d %H:%M:%S')}")

elapsed_time = end_time - start_time
print(f"Ran for minutes: {elapsed_time.total_seconds() / 60:.2f}")

#print(f"Script took {elapsed_time.resolution}  to run.")


----Summary---
 EAST  Copies  EAST Counts Minus 5  Surplus
    6  659760                    1   109960
    7  426493                    2   121855
    8  235280                    3    88230
    9  116104                    4    51601
   10   52761                    5    26380
   11   19352                    6    10555
   12    4539                    7     2647
   13     963                    8      592
   14     173                    9      111
   15      31                   10       20
   16      14                   11        9
Overall, 236101 titles representing 1515470 copies had more than 5 copies

Total number of surplus copies according to countsdf  : 411960 <= will be wrong if some OCN/Matchkey count mismatches
Total number of surplus copies according to allocation: 440406
Total number that went to deaccession - should be same as surplus: 441448

Len deaccessdf: 432389 should be the same as surplus copies plus tier 3: 9199
TIER 3 total - not assigning 9199

Started at: 2